# Data Profiler (con ayuda)

In [1]:
from sqlalchemy import create_engine
import pandas as pd


engine = create_engine('mysql+pymysql://root:@localhost/derivados') # connect to server
conn = engine.connect()

data = pd.read_sql('sp500_earnings', conn)
data

,symbol,EarningsDate,EarningsAverage,EarningsLow,EarningsHigh,RevenueAverage,RevenueLow,RevenueHigh,id
0,CTVA,2021-02-03,-0.06,-0.2,-0.01,2.945860e+09,2.844000e+09,3.038000e+09,1
1,MMM,2021-01-26,2.24,2.14,2.34,8.258220e+09,8.153100e+09,8.356000e+09,2
2,PLD,2021-01-20,0.37,0.33,0.43,9.924300e+08,9.854700e+08,1.001400e+09,3
3,INTU,2020-11-19,0.37,0.12,0.53,1.206510e+09,1.162000e+09,1.226410e+09,4
4,BK,2021-01-20,0.89,0.8,0.96,3.805990e+09,3.750000e+09,3.858000e+09,5
...,...,...,...,...,...,...,...,...,...
498,MU,2021-01-07,0.47,0.46,0.51,5.239970e+09,5.190450e+09,5.372180e+09,499
499,PG,2021-01-21,1.49,1.38,1.58,1.908650e+10,1.864550e+10,1.938700e+10,500
500,IFF,2020-11-09,1.46,1.37,1.53,1.284390e+09,1.260900e+09,1.304360e+09,501
501,GLW,2021-01-27,0.47,0.43,0.5,3.147370e+09,3.089000e+09,3.195000e+09,502


In [2]:
# conda install pandas-profiling
# conda install -c conda-forge/label/cf202003 pandas-profiling

import pandas_profiling as ppf

In [3]:
ppf.ProfileReport(data)

In [4]:
data_daily = pd.read_sql('SELECT * FROM sp500_daily WHERE symbol = "AAPL"', conn)
data_daily

,Date,Close,High,Low,Open,Volume,symbol,variacion,sigma_40,sigma_120,sigma_250,vol_mln,id
0,2018-12-31,38.59,38.98,38.28,38.78,140014000.0,AAPL,0.97,43.73,33.05,28.63,5402.45,21531
1,2019-01-02,38.63,38.86,37.73,37.89,148158800.0,AAPL,0.11,43.40,33.01,28.63,5723.24,21532
2,2019-01-03,34.78,35.64,34.73,35.22,365248800.0,AAPL,-9.96,46.92,35.83,30.30,12703.85,21533
3,2019-01-04,36.27,36.34,35.18,35.35,234428400.0,AAPL,4.27,48.36,36.41,30.59,8501.81,21534
4,2019-01-07,36.19,36.41,35.69,36.37,219111200.0,AAPL,-0.22,48.16,36.41,30.59,7928.63,21535
...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,2020-11-02,108.77,110.68,107.32,109.11,122866900.0,AAPL,-0.08,45.10,40.01,45.71,13364.23,21995
465,2020-11-03,110.44,111.49,108.73,109.66,107624400.0,AAPL,1.54,42.09,40.05,45.73,11886.04,21996
466,2020-11-04,114.95,115.59,112.35,114.14,138235500.0,AAPL,4.08,42.15,40.39,45.88,15890.17,21997
467,2020-11-05,119.03,119.62,116.87,117.95,125734400.0,AAPL,3.55,42.23,40.55,46.00,14966.17,21998


In [5]:
%%time
ppf.ProfileReport(data_daily)

Wall time: 15.3 s


# Threading vs Paralelismo

Tiempos de procesamiento de datos

    •	CPU L1 Cache: 0.9 ns
    •	RAM: 51 ns
    •	SSD: 0.08 a 0.15 ms = 80.000 a 150.000 ns 
    •	HDD 7.2k: 4 ms = 4.000.000 ns
    •	Red: 400 ms = 400.000.000 ns

Unidad equivalente 0.9 nanosegunos computacionales = 1 segundo humano

    •	CPU: 1 seg
    •	RAM: 1 minuto
    •	SSD: 1 a 2 dias
    •	HDD: 1mes y medio
    •	Red: 14 años

# Ejercicio Modelo

## Ejemplo long/short vega

Condiciones


* Que presenten balances en los proximos 7 dias
* Que tengan Sigma diaria de las ultimas 40 ruedas menor/mayor a la de las ultimas 120 (mean reversion)
* Que la volatilidad anualizada HF, PICO en ultimos 15 dias, >+30% / <-30%  al sigma 250 ruedas
* Que la volatilidad anualizada de las velas de 1min del ultimo dia sea  >+10% / <-10% al sigma 40 ruedas


Screener para los tickers que cumplan las condiciones:

* Traer para cada ticker, los 5 contratos de menor/mayor VI
* Traer para cada ticker, los 5 contratos de mayor vega

# Importaciones y conexion a la base de datos

In [1]:
#!pip install PyMySQL

import yfinance as yf
import requests
import pandas as pd
import tqdm, sys, time, threading
import numpy as np
import datetime
from sqlalchemy import create_engine

In [4]:
engine = create_engine('mysql+pymysql://root:@localhost') # connect to server
engine.execute("CREATE DATABASE If NOT EXISTS derivados") 

In [5]:
engine = create_engine('mysql+pymysql://root:@localhost/derivados') # connect to server
conn = engine.connect()

# Tickers del SP500

In [4]:
sp500_wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500_wiki = sp500_wiki.loc[~sp500_wiki.Symbol.isin(['BRK.B' ,'BF.B'])]
sp500_wiki

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [5]:
sp500_wiki.to_sql(con=conn, name='sp500_tickers', if_exists='replace')

In [6]:
q = 'ALTER TABLE sp500_tickers MODIFY COLUMN Symbol VARCHAR(255)'
conn.execute(q)

In [7]:
q = 'ALTER TABLE sp500_tickers ADD PRIMARY KEY(Symbol)'
conn.execute(q)

In [8]:
%%time
tickers = pd.read_sql('SELECT Symbol FROM sp500_tickers', conn)

Wall time: 18.9 ms


In [9]:
sp500_tickers = tickers.Symbol.tolist()

In [11]:
sp500_tickers[:10]

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES']

# Historicos Yahoo Diarios

In [12]:
def getDataM(listado, start='2000-01-01', interval='1d', end=None):
    data = yf.download(listado, start=start, end=end, interval=interval, auto_adjust=True)
    return data.swaplevel(i=1, j=0, axis=1)

## Descarga de datos

In [13]:
%%time
df_sp500 = getDataM(sp500_tickers, start='2018-01-01', interval='1d')

[*********************100%***********************]  503 of 503 completed
Wall time: 43.7 s


## Preparamos la data

In [15]:
tablas = []
with tqdm.tqdm(total=len(sp500_tickers), file=sys.stdout) as pbar:
    for ticker in sp500_tickers:
        pbar.update()
        tabla = df_sp500[ticker].copy()
        tabla['symbol'] = ticker
        tabla['variacion'] = tabla.Close.pct_change() *100
        tabla['sigma_40'] = tabla.variacion.rolling(40).std() * 250**0.5
        tabla['sigma_120'] = tabla.variacion.rolling(120).std() * 250**0.5
        tabla['sigma_250'] = tabla.variacion.rolling(250).std() * 250**0.5
        tabla['vol_mln'] = tabla.Volume * tabla.Close / 1000000
        tabla = tabla.dropna().round(2)
        tablas.append(tabla)

tabla_daily = pd.concat(tablas)        
tabla_daily

100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [00:06<00:00, 79.51it/s]


,Close,High,Low,Open,Volume,symbol,variacion,sigma_40,sigma_120,sigma_250,vol_mln
Date,,,,,,,,,,,
2018-12-31,179.25,180.29,177.33,179.06,1804400.0,MMM,0.62,28.73,23.99,23.83,323.44
2019-01-02,179.64,179.67,175.64,176.69,2475200.0,MMM,0.22,28.62,23.84,23.84,444.63
2019-01-03,172.87,177.12,172.05,177.12,3358200.0,MMM,-3.77,30.12,24.42,24.08,580.54
2019-01-04,179.98,180.60,175.01,175.68,2995100.0,MMM,4.11,31.90,25.11,24.43,539.07
2019-01-07,179.57,180.91,177.48,180.02,2162200.0,MMM,-0.23,31.79,25.11,24.43,388.26
...,...,...,...,...,...,...,...,...,...,...,...
2020-11-02,162.89,163.26,160.71,161.31,1516600.0,ZTS,2.74,22.44,24.14,38.38,247.04
2020-11-03,163.14,165.49,162.90,164.64,1519500.0,ZTS,0.15,21.52,23.93,38.37,247.89
2020-11-04,174.35,176.17,165.97,165.99,3403700.0,ZTS,6.87,26.03,25.74,38.95,593.44


## Guardamos la tabla

In [16]:
%%time
tabla_daily.to_sql(con=conn,  name='sp500_daily', if_exists='replace')

Wall time: 15.4 s


## Agregamos una PK (recomendable, no necesario)

In [17]:
q = 'ALTER TABLE sp500_daily ADD id INT NOT NULL AUTO_INCREMENT PRIMARY KEY'
conn.execute(q)

## Algunas consultas

In [18]:
%%time
pd.read_sql('''SELECT symbol, AVG(sigma_40) AS Sigma_40_AVG , AVG(sigma_250) AS Sigma_250_AVG
                FROM sp500_daily 
                WHERE Date >"2020-06-01" 
                GROUP BY symbol'''
            , conn)

Wall time: 311 ms


,symbol,Sigma_40_AVG,Sigma_250_AVG
0,A,23.970714,36.082768
1,AAL,95.459464,98.634107
2,AAP,28.839107,47.082857
3,AAPL,37.884643,42.398036
4,ABBV,22.970625,33.952589
...,...,...,...
493,YUM,23.994286,43.519018
494,ZBH,37.157411,47.227589
495,ZBRA,37.325000,49.626607
496,ZION,52.068304,49.375536


In [19]:
%%time
pd.read_sql('''SELECT symbol, STDDEV(variacion)*SQRT(250) AS Sigma
                FROM sp500_daily 
                WHERE Date >"2020-06-01" 
                GROUP BY symbol'''
            , conn)

Wall time: 310 ms


,symbol,Sigma
0,A,22.478738
1,AAL,96.781115
2,AAP,26.582093
3,AAPL,41.622950
4,ABBV,25.355542
...,...,...
493,YUM,22.562429
494,ZBH,34.194850
495,ZBRA,36.649809
496,ZION,50.377656


In [20]:
%%time
pd.read_sql('''SELECT symbol, STDDEV(variacion)*SQRT(250) AS Sigma, 
                        AVG(sigma_40) AS Sigma_40_AVG , 
                        AVG(sigma_250) AS Sigma_250_AVG
                FROM sp500_daily 
                WHERE Date >"2020-06-01" 
                GROUP BY symbol'''
            , conn).round(1)

Wall time: 314 ms


,symbol,Sigma,Sigma_40_AVG,Sigma_250_AVG
0,A,22.5,24.0,36.1
1,AAL,96.8,95.5,98.6
2,AAP,26.6,28.8,47.1
3,AAPL,41.6,37.9,42.4
4,ABBV,25.4,23.0,34.0
...,...,...,...,...
493,YUM,22.6,24.0,43.5
494,ZBH,34.2,37.2,47.2
495,ZBRA,36.6,37.3,49.6
496,ZION,50.4,52.1,49.4


# Historicos Yahoo 5min

In [21]:
%%time
start = (datetime.date.today() - datetime.timedelta(days=30)).isoformat()
df_sp500_5m = getDataM(sp500_tickers, start=start, interval='5m')

[*********************100%***********************]  503 of 503 completed
Wall time: 46.3 s


## Preparamos la data

In [23]:
6.5*(60/5)*5

390.0

In [24]:
tablas = []
with tqdm.tqdm(total=len(sp500_tickers), file=sys.stdout) as pbar:
    for ticker in sp500_tickers:
        pbar.update()
        tabla = df_sp500_5m[ticker].copy()
        tabla['symbol'] = ticker
        tabla['variacion'] = tabla.Close.pct_change() *100
        tabla['sigma_1d'] = tabla.variacion.rolling(78).std() * (78 * 250)**0.5 
        tabla['sigma_5d'] = tabla.variacion.rolling(390).std() * (390 * 250)**0.5
        tabla['vol_mln'] = tabla.Volume * tabla.Close / 1000000
        tabla = tabla.dropna().round(2)
        tablas.append(tabla)

tabla_5m = pd.concat(tablas)
tabla_5m

100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [00:05<00:00, 89.75it/s]


,Close,High,Low,Open,Volume,symbol,variacion,sigma_1d,sigma_5d,vol_mln
Datetime,,,,,,,,,,
2020-10-14 09:30:00-04:00,167.82,168.35,166.79,167.01,64605.0,MMM,0.71,21.78,41.94,10.84
2020-10-14 09:35:00-04:00,167.89,168.05,167.59,167.74,13647.0,MMM,0.04,21.76,41.85,2.29
2020-10-14 09:40:00-04:00,168.24,168.24,167.78,167.78,12299.0,MMM,0.21,21.74,41.97,2.07
2020-10-14 09:45:00-04:00,168.51,168.55,167.93,168.10,32092.0,MMM,0.16,21.83,41.78,5.41
2020-10-14 09:50:00-04:00,168.60,168.83,168.50,168.54,18726.0,MMM,0.05,21.62,41.79,3.16
...,...,...,...,...,...,...,...,...,...,...
2020-11-06 15:35:00-05:00,169.49,169.61,169.39,169.50,38676.0,ZTS,-0.01,36.03,107.27,6.56
2020-11-06 15:40:00-05:00,169.72,170.09,169.56,169.56,26998.0,ZTS,0.14,36.08,107.15,4.58
2020-11-06 15:45:00-05:00,170.00,170.12,169.73,169.77,31206.0,ZTS,0.16,36.15,107.09,5.31


## Guardamos la tabla

In [25]:
%%time
tabla_5m.to_sql(con=conn,  name='sp500_5m', if_exists='replace')

Wall time: 36.9 s


## Agregamos una PK (recomendable, no necesario)

In [26]:
q = 'ALTER TABLE sp500_5m ADD id INT NOT NULL AUTO_INCREMENT PRIMARY KEY'
conn.execute(q)

## Algunas Consultas

Volatilidades de alta fracuencia:
   
Para los ultimos 10 (n) dias:
 * La volatilidad anualizada de los movimientos cada 5min
 * El pico de un rolling_1_dia del std() anualizado de las velas de 5min 
 * El pico de un rolling_5_dias del std() anualizado de las velas de 5min

In [28]:
%%time
pd.read_sql('''SELECT symbol, STDDEV(variacion)*SQRT(250*78) AS Sigma_10d, 
                        MAX(sigma_1d) AS pico_sigma_r1d , 
                        MAX(sigma_5d) AS pico_sigma_r5d
                FROM sp500_5m 
                WHERE Datetime > DATE_SUB(NOW(), INTERVAL 10 day) 
                GROUP BY symbol'''
            , conn).round(1)

Wall time: 1.41 s


,symbol,Sigma_10d,pico_sigma_r1d,pico_sigma_r5d
0,A,33.5,42.9,80.4
1,AAL,51.2,80.0,135.4
2,AAP,37.3,49.4,91.3
3,AAPL,46.1,66.8,109.9
4,ABBV,70.9,157.0,193.2
...,...,...,...,...
498,YUM,37.2,55.4,91.8
499,ZBH,48.7,87.4,116.7
500,ZBRA,55.5,117.5,143.4
501,ZION,72.4,165.7,189.6


# Calendario Yahoo

## Sin hilos

In [30]:
tickers = pd.read_sql('SELECT symbol FROM sp500_tickers', conn).symbol.tolist()
tickers = tickers[:12]

In [45]:
dataCal = yf.Ticker('TSLA').calendar
dataCal.columns = ['juan','pablo']
dataCal

,juan,pablo
Earnings Date,2021-01-27 00:00:00,2021-02-01 00:00:00
Earnings Average,0.87,0.87
Earnings Low,0.46,0.46
Earnings High,1.16,1.16
Revenue Average,10023400000,10023400000
Revenue Low,9050000000,9050000000
Revenue High,10683300000,10683300000


In [49]:
dataCal.iloc[:,1].to_dict()

{'Earnings Date': Timestamp('2021-02-01 00:00:00'),
 'Earnings Average': 0.87,
 'Earnings Low': 0.46,
 'Earnings High': 1.16,
 'Revenue Average': 10023400000,
 'Revenue Low': 9050000000,
 'Revenue High': 10683300000}

In [31]:
tickers

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AFL',
 'A']

In [50]:
%%time
earnings = []
for ticker in tickers:
    try:
        data = yf.Ticker(ticker)
        dicc = data.calendar.iloc[:,0].to_dict()
        dicc['symbol'] = ticker
        earnings.append(dicc)
        print(ticker, end=' ')
    except:
        print(f'\nNo se pudo usar data de {ticker}\n')        
        
sp500_earnings = pd.DataFrame(earnings)
sp500_earnings

MMM ABT ABBV ABMD ACN ATVI ADBE AMD AAP AES AFL A Wall time: 38.3 s


,Earnings Date,Earnings Average,Earnings Low,Earnings High,Revenue Average,Revenue Low,Revenue High,symbol
0,2021-01-26,2.24,2.14,2.34,8258220000,8153100000,8356000000,MMM
1,2021-01-20,1.36,1.32,1.40,9969880000,9764000000,10299600000,ABT
2,2021-02-05,2.86,2.83,2.90,13746300000,13671000000,13821500000,ABBV
3,2021-02-04,1.12,1.01,1.21,227800000,221000000,243200000,ABMD
4,2020-12-17,2.05,1.93,2.22,11366500000,11186700000,11529000000,ACN
5,2021-02-04,1.17,1.09,1.45,2799380000,2716000000,3108000000,ATVI
6,2020-12-09,2.66,2.63,2.82,3359350000,3347000000,3404000000,ADBE
7,2021-01-26,0.46,0.44,0.49,3012110000,2994000000,3093270000,AMD
8,2020-11-10,2.66,2.14,3.10,2476080000,2421900000,2557000000,AAP
9,2020-11-06,0.43,0.41,0.45,2701240000,2649000000,2753470000,AES


Cuellos de Botella


* Procesador (Calculos)
* Memoria (Datos)
* Disco (Almacenamiento)
* Red (Trafico)  >> Hilos


In [ ]:
GIL

## Con Hilos prueba

In [52]:
def get_earnings(ticker):
    try:
        print(ticker, end=' ')
        data = yf.Ticker(ticker)
        dicc = data.calendar.iloc[:,0].to_dict()
        dicc['symbol'] = ticker
        return pd.DataFrame.from_dict(dicc, orient='index').T
    except:
        print(f'\nNo se pudo usar data de {ticker}\n')
        columns = ['Earnings Date','Earnings Average', 'Earnings Low', 'Earnings High', 
                   'Revenue Average', 'Revenue Low', 'Revenue High', 'symbol']
        df = pd.DataFrame(columns=columns)
        df.loc[0,'symbol'] = ticker
        return df

get_earnings('AAPL')

AAPL 

,Earnings Date,Earnings Average,Earnings Low,Earnings High,Revenue Average,Revenue Low,Revenue High,symbol
0,2021-01-26,1.39,1.23,1.58,101875000000,97739000000,110875000000,AAPL


In [53]:
tickers

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AFL',
 'A']

In [55]:
n_threads = 4
subs = np.array_split(tickers, n_threads)
subs

[array(['MMM', 'ABT', 'ABBV'], dtype='<U4'),
 array(['ABMD', 'ACN', 'ATVI'], dtype='<U4'),
 array(['ADBE', 'AMD', 'AAP'], dtype='<U4'),
 array(['AES', 'AFL', 'A'], dtype='<U4')]

In [57]:
%%time

dfs = []
def worker(tickers):
    for ticker in tickers:
        df  = get_earnings(ticker)
        dfs.append(df)
    return df

    
threads = []
for i in range(n_threads):
    t = threading.Thread(target=worker, args=(subs[i],))
    threads.append(t)
    t.start()
    
for t in threads:
    t.join()
    
sp500_earnings = pd.concat(dfs)
sp500_earnings

MMM ABMD ADBE AES ABT AFL AMD ACN AAAP  ABBV ATVI Wall time: 10.1 s


,Earnings Date,Earnings Average,Earnings Low,Earnings High,Revenue Average,Revenue Low,Revenue High,symbol
0,2021-01-26,2.24,2.14,2.34,8258220000,8153100000,8356000000,MMM
0,2020-11-06,0.43,0.41,0.45,2701240000,2649000000,2753470000,AES
0,2020-12-09,2.66,2.63,2.82,3359350000,3347000000,3404000000,ADBE
0,2021-02-04,1.12,1.01,1.21,227800000,221000000,243200000,ABMD
0,2021-02-02,1.05,1.01,1.07,5506460000,5505000000,5507910000,AFL
0,2021-01-26,0.46,0.44,0.49,3012110000,2994000000,3093270000,AMD
0,2021-01-20,1.36,1.32,1.4,9969880000,9764000000,10299600000,ABT
0,2020-12-17,2.05,1.93,2.22,11366500000,11186700000,11529000000,ACN
0,2020-11-10,2.66,2.14,3.1,2476080000,2421900000,2557000000,AAP
0,2020-11-23,0.93,0.85,1.09,1403890000,1386900000,1423890000,A


## Con hilos full

RATE LIMIT


* Binance: 20 y 30 Req x Seg
* HitBTC: entre 70 y 100 req x seg
* IEX Cloud: 60 Req x Seg  (puede que un poco mas, tipo 80)

    
1 request tipico puede tardar 400 ms

In [58]:
tickers = pd.read_sql('SELECT symbol FROM sp500_tickers', conn).symbol.tolist()

In [59]:
n_threads = 8
subs = np.array_split(tickers, n_threads)

In [60]:
%%time

dfs = []
def worker(tickers):
    for ticker in tickers:
        df  = get_earnings(ticker)
        dfs.append(df)
    return df

    
t0 = time.time()
threads = []
for i in range(n_threads):
    t = threading.Thread(target=worker, args=(subs[i],))
    threads.append(t)
    t.start()
    
for t in threads:
    t.join()

print('\n')
sp500_earnings = pd.concat(dfs)
sp500_earnings

MMM BK CTVA FE INTU MSFT PLD TJX COST ABT PRU ISRG BAX MAA FRC TSCO CCI PEG FISV MHK ABBV BDX IVZ TT CSX TAP TDG BBY ABMD PSA IPGP FLT CMI ACN MDLZ PHM BIO TRV IQV FLIR ATVI MNST PVH CVS TFC FLS IRM BIIB 
No se pudo usar data de CVS

DHI ADBE QRVO MCO JKHY BLKTWTR  DHR FMC PWR AMD MS J BA AAP TYL F DRI QCOM MOS BKNG JBHT DVA FTNT AES TSN DGX BWA SJM MSI FTV UDR AFL DE RL BXP MSCI JNJ FBHS ULTA A DAL RJF BSX MYL JCI FOXA APD XRAY USB RTX BMY NDAQ JPM FOX DVN AKAM UAA O AVGO 
No se pudo usar data de FOX

BEN NOV DXCM ALK JNPR UA REG BR FCX FANG ALB NTAP REGN UNP KSU CHRW GPS NFLX K ARE DLR UAL RF COG GRMN DFS NWL ALXN UNH RSG KEY IT CDNS DISCA UPS KEYS ALGN NEM RMD GD CPB DISCK KMB ALLE URI NWSA RHI COF GE KIM DISH LNT UHS NWS ROK DG CAH ALL KMI 
No se pudo usar data de NWS

NEE UNM ROL GIS DLTR KMX GOOGL VFC NLSN KLAC GM ROP D CCL VLO GOOG KHC NKE ROST GPC DPZ CARR 
No se pudo usar data de GOOG

MO VAR KR NI RCL GILD DOV CTLT AMZN VTR LB NSC GL SPGI DOW CAT VRSN AMCR LHX CRM GPN NTRS DT

,Earnings Date,Earnings Average,Earnings Low,Earnings High,Revenue Average,Revenue Low,Revenue High,symbol
0,2021-02-03,-0.06,-0.2,-0.01,2945860000,2844000000,3038000000,CTVA
0,2021-01-26,2.24,2.14,2.34,8258220000,8153100000,8356000000,MMM
0,2021-01-20,0.37,0.33,0.43,992430000,985470000,1001400000,PLD
0,2020-11-19,0.37,0.12,0.53,1206510000,1162000000,1226410000,INTU
0,2021-01-20,0.89,0.8,0.96,3805990000,3750000000,3858000000,BK
...,...,...,...,...,...,...,...,...
0,2021-01-07,0.47,0.46,0.51,5239970000,5190450000,5372180000,MU
0,2021-01-21,1.49,1.38,1.58,19086500000,18645500000,19387000000,PG
0,2020-11-09,1.46,1.37,1.53,1284390000,1260900000,1304360000,IFF
0,2021-01-27,0.47,0.43,0.5,3147370000,3089000000,3195000000,GLW


In [61]:
cols = list(sp500_earnings.columns)
cols = [c.replace(' ','') for c in cols]
sp500_earnings.columns = cols

In [62]:
sp500_earnings.set_index('symbol', inplace=True)

In [63]:
%%time
sp500_earnings.to_sql('sp500_earnings', conn, if_exists='replace')

Wall time: 71.8 ms


In [64]:
q = 'ALTER TABLE sp500_earnings ADD id INT NOT NULL AUTO_INCREMENT PRIMARY KEY'
conn.execute(q)

## Algunas consultas

In [48]:
hoy = datetime.date.today().isoformat()

fecha_limite = datetime.date.today() + datetime.timedelta(days=15)
fecha_limite = fecha_limite.isoformat()

q = f'''SELECT symbol FROM sp500_earnings 
        WHERE EarningsDate < "{fecha_limite}" AND EarningsDate > "{hoy}"   '''

pd.read_sql(q, conn)

,symbol
0,TJX
1,BDX
2,TDG
3,FLT
4,FLS
...,...
79,MCHP
80,ZTS
81,BLL
82,CPRT


# Opciones desde TDA

In [65]:
from clase_11_keys import *
c_key = TDA_KEY

In [50]:
def options(symbol):
    params = {'apikey' : c_key, 'symbol':symbol}
    endpoint = 'https://api.tdameritrade.com/v1/marketdata/chains'
    r = requests.get(url=endpoint ,params=params)
    return r.json()

In [51]:
ticker = 'GGAL'
chain = options(ticker)
chain

{'symbol': 'GGAL',
 'status': 'SUCCESS',
 'underlying': None,
 'strategy': 'SINGLE',
 'interval': 0.0,
 'isDelayed': True,
 'isIndex': False,
 'interestRate': 0.1,
 'underlyingPrice': 7.045,
 'volatility': 29.0,
 'daysToExpiration': 0.0,
 'numberOfContracts': 104,
 'putExpDateMap': {'2020-11-20:16': {'2.5': [{'putCall': 'PUT',
     'symbol': 'GGAL_112020P2.5',
     'description': 'GGAL Nov 20 2020 2.5 Put',
     'exchangeName': 'OPR',
     'bid': 0.0,
     'ask': 0.05,
     'last': 0.0,
     'mark': 0.03,
     'bidSize': 0,
     'askSize': 124,
     'bidAskSize': '0X124',
     'lastSize': 0,
     'highPrice': 0.0,
     'lowPrice': 0.0,
     'openPrice': 0.0,
     'closePrice': 0.0,
     'totalVolume': 0,
     'tradeDate': None,
     'tradeTimeInLong': 0,
     'quoteTimeInLong': 1604515313242,
     'netChange': 0.0,
     'volatility': 253.785,
     'delta': -0.015,
     'gamma': 0.01,
     'theta': -0.004,
     'vega': 0.001,
     'rho': 0.0,
     'openInterest': 0,
     'timeValue': 0.

## Metemos el OptionChain en un DataFrame

In [52]:
ticker = 'AAPL'
chain = options(ticker)

v_calls = list(chain['callExpDateMap'].values())
v_calls_fechas = list(chain['callExpDateMap'].keys())
v_puts = list(chain['putExpDateMap'].values())
v_puts_fechas = list(chain['putExpDateMap'].keys())
calls = []
for i in range(len(v_calls)):
    v = list(v_calls[i].values())    
    for j in range(len(v)):
        calls.append(v[j][0])

puts = []
for i in range(len(v_puts)):
    v = list(v_puts[i].values())    
    for j in range(len(v)):
        puts.append(v[j][0])
        
contracts = pd.concat([pd.DataFrame(calls),pd.DataFrame(puts)])

tabla = contracts.loc[contracts.daysToExpiration>0]
tabla = tabla.loc[:,['symbol','strikePrice','daysToExpiration','putCall','bid','ask',
                     'last','volatility','openInterest','theoreticalOptionValue',
                     'delta', 'gamma', 'theta', 'vega', 'rho', 'inTheMoney']]

tabla.columns = ['symbol_opc','Strike','TTM','Type','Bid','Ask','Last','IV','OpenInt','Theor',
                 'delta', 'gamma', 'theta', 'vega', 'rho','ITM']

tabla['symbol'] = ticker
tabla

,symbol_opc,Strike,TTM,Type,Bid,Ask,Last,IV,OpenInt,Theor,delta,gamma,theta,vega,rho,ITM,symbol
0,AAPL_110620C60,60.0,2,CALL,54.90,55.00,55.00,NaN,174,NaN,NaN,NaN,NaN,0.009,NaN,True,AAPL
1,AAPL_110620C65,65.0,2,CALL,49.90,50.00,50.24,NaN,127,NaN,NaN,NaN,NaN,0.009,NaN,True,AAPL
2,AAPL_110620C70,70.0,2,CALL,44.90,45.00,45.31,NaN,27,NaN,NaN,NaN,NaN,0.010,NaN,True,AAPL
3,AAPL_110620C71,71.0,2,CALL,43.90,44.00,39.15,NaN,1,NaN,NaN,NaN,NaN,0.010,NaN,True,AAPL
4,AAPL_110620C72,72.0,2,CALL,42.90,43.00,36.75,NaN,28,NaN,NaN,NaN,NaN,0.010,NaN,True,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232,AAPL_012023P190,190.0,807,PUT,80.20,81.80,80.05,31.36,1,81,-0.796,0.005,-0.01,0.458,-3.816,True,AAPL
1233,AAPL_012023P195,195.0,807,PUT,84.70,86.35,85.35,31.426,1,85.525,-0.81,0.005,-0.01,0.438,-3.951,True,AAPL
1234,AAPL_012023P200,200.0,807,PUT,89.25,90.95,96.05,31.365,10,90.1,-0.824,0.005,-0.009,0.415,-4.088,True,AAPL
1235,AAPL_012023P210,210.0,807,PUT,98.50,100.15,105.35,30.94,11,99.325,-0.853,0.004,-0.008,0.363,-4.365,True,AAPL


## Encapsulamos el DF en una función

In [53]:
def optionsDF(chain):
    try:
        v_calls = list(chain['callExpDateMap'].values())
        v_calls_fechas = list(chain['callExpDateMap'].keys())
        v_puts = list(chain['putExpDateMap'].values())
        v_puts_fechas = list(chain['putExpDateMap'].keys())
        calls = []
        for i in range(len(v_calls)):
            v = list(v_calls[i].values())    
            for j in range(len(v)):
                calls.append(v[j][0])

        puts = []
        for i in range(len(v_puts)):
            v = list(v_puts[i].values())    
            for j in range(len(v)):
                puts.append(v[j][0])

        contracts = pd.concat([pd.DataFrame(calls),pd.DataFrame(puts)])
        tabla = contracts.loc[contracts.daysToExpiration>0]
        tabla = tabla.loc[:,['symbol','strikePrice','daysToExpiration','putCall','bid','ask',
                             'last','volatility','openInterest','theoreticalOptionValue',
                             'delta', 'gamma', 'theta', 'vega', 'rho', 'inTheMoney']]

        tabla.columns = ['symbol_opc','Strike','TTM','Type','Bid','Ask','Last','IV','OpenInt','Theor',
                         'delta', 'gamma', 'theta', 'vega', 'rho','ITM']

        tabla['symbol'] = chain['symbol']
    except:
        tabla = pd.DataFrame()
        
    return tabla

In [54]:
ticker = 'FB'
df = optionsDF(options(ticker))
df

,symbol_opc,Strike,TTM,Type,Bid,Ask,Last,IV,OpenInt,Theor,delta,gamma,theta,vega,rho,ITM,symbol
0,FB_110620C135,135.0,2,CALL,152.30,153.15,145.65,NaN,22,NaN,NaN,NaN,NaN,0.001,NaN,True,FB
1,FB_110620C140,140.0,2,CALL,147.20,148.15,123.65,NaN,7,NaN,NaN,NaN,NaN,0.001,NaN,True,FB
2,FB_110620C145,145.0,2,CALL,142.25,143.10,119.85,223.44,13,142.675,1,0,-0.009,0.000,0.012,True,FB
3,FB_110620C150,150.0,2,CALL,137.20,138.15,136.37,212.719,12,137.675,1,0,-0.009,0.000,0.012,True,FB
4,FB_110620C155,155.0,2,CALL,132.30,133.05,109.85,202.356,12,132.675,1,0,-0.009,0.000,0.013,True,FB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204,FB_012023P480,480.0,807,PUT,201.60,205.90,210.49,33.87,6,203.75,-0.735,0.001,-0.025,1.236,-4.992,True,FB
1205,FB_012023P500,500.0,807,PUT,219.10,222.70,226.47,33.663,15,220.9,-0.752,0.001,-0.024,1.158,-4.788,True,FB
1206,FB_012023P520,520.0,807,PUT,237.05,241.45,0.00,34.405,0,239.25,-0.756,0.001,-0.023,1.117,-4.628,True,FB
1207,FB_012023P540,540.0,807,PUT,255.50,259.90,264.95,35.231,5,257.7,-0.756,0.001,-0.023,1.086,-4.459,True,FB


## Bckp opciones sp500 en la BBDD (Threading)

In [55]:
tickers = pd.read_sql('SELECT symbol FROM sp500_tickers', conn).symbol.tolist()
n_threads = 2
subs = np.array_split(tickers, n_threads)

In [56]:
%%time

dfs = []
def worker(tickers):
    for ticker in tickers:
        print(ticker, end=' ')
        df = optionsDF(options(ticker))
        dfs.append(df)
        df.to_sql('sp500_opciones', engine, if_exists='append')
    return df

    
t0 = time.time()
threads = []
for i in range(n_threads):
    t = threading.Thread(target=worker, args=(subs[i],))
    threads.append(t)
    t.start()
    
for t in threads:
    t.join()
    
result = pd.concat(dfs)
result

MMM INTU ISRG ABT ABBV IVZ IPGP ABMD IQV ACN IRM JKHY ATVI J JBHT ADBE SJM AMD JNJ AAP AES JCI AFL JPM A JNPR APD KSU AKAM K ALK KEY ALB KEYS ARE ALXN KMB KIM ALGN KMI ALLE KLAC LNT ALL GOOGL KHC GOOG KR LB MO LHX AMZN LH AMCR LRCX AEE AAL LW LVS AEP LEG AXP LDOS LEN AIG LLY AMT LNC AWK AMP LIN ABC LYV LKQ AME LMT AMGN L APH LOW ADI LUMN LYB ANSS ANTM MTB AON MRO AOS MPC APA MKTX AIV MAR AAPL MMC AMAT MLM MAS MA APTV ADM ANET MKC AJG MXIM AIZ MCD T MCK ATO ADSK MDT ADP MRK AZO MET AVB MTD AVY MGM BKR MCHP BLL BAC MU BK MSFT BAX MAA BDX MHK TAP BBY MDLZ BIO MNST MCO BIIB MS BLK MOS MSI BA MSCI MYL BKNG NDAQ NOV BWA BXP NTAP BSX NFLX BMY NWL AVGO NEM NWSA BR NWS NEE CHRW NLSN COG NKE CDNS NI CPB NSC COF CAH KMX NTRS NOC CCL CARR NLOK CTLT NCLH CAT NRG CBOE NUE CBRE NVDA CDW NVR ORLY CE OXY CNC ODFL CNP OMC CERN OKE CF ORCL SCHW OTIS CHTR PCAR PKG CVX PH PAYX CMG PAYC CB PYPL CHD CI PNR CINF PBCT CTAS CSCO PEP C PKI CFG PRGO CTXS PFE CLX PM CME PSX CMS PNW KO PXD PNC CTSH POOL CL PPG PPL 

,symbol_opc,Strike,TTM,Type,Bid,Ask,Last,IV,OpenInt,Theor,delta,gamma,theta,vega,rho,ITM,symbol
0,INTU_110620C165,165.0,2,CALL,177.6,181.6,0.00,NaN,0,NaN,NaN,NaN,NaN,0.036,NaN,True,INTU
1,INTU_110620C170,170.0,2,CALL,173.0,177.5,0.00,5,0,175.25,1,0,0,0.088,0,True,INTU
2,INTU_110620C175,175.0,2,CALL,168.2,172.7,0.00,540.577,0,170.45,0.886,0,-2.44,0.036,0.002,True,INTU
3,INTU_110620C180,180.0,2,CALL,163.8,167.3,0.00,514.893,0,165.55,0.889,0,-3.507,0.037,0.002,True,INTU
4,INTU_110620C185,185.0,2,CALL,158.3,162.8,0.00,484.083,0,160.55,0.891,0,-2.838,0.037,0.002,True,INTU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,IFF_012023P165,165.0,807,PUT,68.0,71.9,0.00,31.888,0,69.85,-0.748,0.005,-0.014,0.416,-3.294,True,IFF
193,IFF_012023P170,170.0,807,PUT,72.4,76.3,0.00,31.843,0,74.35,-0.764,0.005,-0.014,0.393,-3.433,True,IFF
194,IFF_012023P175,175.0,807,PUT,77.3,80.6,68.24,31.878,3,78.95,-0.779,0.005,-0.013,0.372,-3.568,True,IFF
195,IFF_012023P180,180.0,807,PUT,81.7,85.7,72.27,31.936,1,83.6,-0.792,0.004,-0.013,0.352,-3.702,True,IFF


## Arreglo BUG

TDA algunas veces tira IV = -999

In [57]:
query = 'UPDATE sp500_opciones SET IV = NULL WHERE IV < 0'
r = engine.execute(query)

## Agrego PK

In [58]:
q = 'ALTER TABLE sp500_opciones ADD id INT NOT NULL AUTO_INCREMENT PRIMARY KEY'
conn.execute(q)

In [59]:
q = 'ALTER TABLE sp500_opciones CHANGE IV IV FLOAT NULL DEFAULT NULL'
conn.execute(q)

# Intraday 1min Sigma

In [67]:
from clase_11_keys import *
iex_token = IEX_TOKEN

In [68]:
def iex_intraday(symbol):
    url = f'https://cloud.iexapis.com/stable/stock/{symbol}/intraday-prices'
    params = {'chartIEXOnly':True, 'token':iex_token }
    js = requests.get(url, params=params).json()
    df = pd.DataFrame(js)
    df.date = pd.to_datetime(df.date)
    df['symbol'] = symbol
    df['pctChange'] = df.average.pct_change()*100
    return df.dropna().round(3)

## Descarga con hilos

In [69]:
tickers = pd.read_sql('SELECT symbol FROM sp500_tickers', conn).symbol.tolist()
n_threads = 40
subs = np.array_split(tickers, n_threads)

In [70]:
%%time

dfs = []
def worker(tickers):
    for ticker in tickers:
        print(ticker, end=' ')
        df = iex_intraday(ticker)
        dfs.append(df)
    return df

    
t0 = time.time()
threads = []
for i in range(n_threads):
    t = threading.Thread(target=worker, args=(subs[i],))
    threads.append(t)
    t.start()
    
for t in threads:
    t.join()
    
result = pd.concat(dfs)
result

MMM AKAM AMCR ADI AIZ BDX CHRW CDW CINF CMACMI  DFS DD EQIX FFIV FMC GE HCA HWM ICE JBHT KLAC LNC MAR MTDMOS  NEE NVR PAYC PNC PVH RSG STX SBUX TGT TT UNP VRSKDIS  WMB ABTAEE  CVS ANSSALK FB  PYPL DXC CE DISCA LINPEAK  STT IBM T KHCMSI  CAG BBYTDGFVZ ORLY  GIS EQR  CTAS QRVO TEL  SEEHPQWM SJM  NLSN  COG UALMMC WLTWRMD   MGMPOOL  AAL ABBV FAST ANTM DISCK CNC PNR ALB EMN STE LYV MSCI JNJ HSIC BIOKRWYNN   DHI TRV MCHP IP FTNTMLM OXY  UNH PPGSRE  CSCOGM  FTI ESSCXOATO  RHI CDNSPWR  WAT  NKE HUM VRTX AEP ABMD AON FRT ARE ETN CNP SYKDISH  LKQ PBCT MYL JCI MU BIIB ODFL TFC HSY LBDHR  IPG WECNOW XEL  ELMAS TDYC   ROK QCOM PPLGPC  NI ADSK CPB UPS FTV COPVIAC  HBAN ACN AXP AOS ALXN FDX DG CERN LMT SIVB PEP EBAY NDAQ MSFT JPM OMC BLK HES TWTR DRI IFF LHX XRX WFC TFX SHW COF MA CFGROLPFG   GILD ETSYURI DGXADPFBHSHII     EDAIG  NSC V ATVI APA FIS ALGN DLTR ECL PKI CF NOVL  SYF MAA JNPR BA OKE TYL HPE INTU LH DVA WELL TER XLNX SPG PG RLCAH  CTXS IEX ROP MKCAZO  GL FOXA UHS STZ FITBAMT EVRG  NTRS AIV

,date,minute,label,high,low,open,close,average,volume,notional,numberOfTrades,symbol,pctChange
1,2020-11-04,09:31,09:31 AM,162.910,162.790,162.880,162.910,162.850,764.0,124417.320,6.0,MMM,-0.368
2,2020-11-04,09:32,09:32 AM,163.135,162.790,162.790,163.135,163.025,927.0,151124.320,13.0,MMM,0.107
3,2020-11-04,09:33,09:33 AM,162.840,162.580,162.810,162.580,162.757,1075.0,174963.500,12.0,MMM,-0.164
4,2020-11-04,09:34,09:34 AM,162.740,162.460,162.510,162.740,162.485,310.0,50370.400,5.0,MMM,-0.167
5,2020-11-04,09:35,09:35 AM,162.240,161.875,162.240,161.875,161.934,850.0,137644.000,9.0,MMM,-0.339
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,2020-11-04,13:55,1:55 PM,150.400,150.340,150.400,150.360,150.385,540.0,81207.900,5.0,LLY,-0.072
266,2020-11-04,13:56,1:56 PM,150.410,150.140,150.405,150.140,150.296,810.0,121739.435,11.0,LLY,-0.059
267,2020-11-04,13:57,1:57 PM,150.310,150.160,150.175,150.160,150.210,2168.0,325655.950,16.0,LLY,-0.057
268,2020-11-04,13:58,1:58 PM,150.060,149.960,150.040,150.060,150.037,992.0,148836.680,9.0,LLY,-0.115


## Guardado en BBDD

In [71]:
%%time
result.to_sql('sp500_iex_intraday', conn, if_exists='replace')

Wall time: 6.78 s


In [72]:
q = 'ALTER TABLE sp500_iex_intraday ADD id INT NOT NULL AUTO_INCREMENT PRIMARY KEY'
conn.execute(q)

In [73]:
%%time
q = '''SELECT symbol, STDDEV(pctChange) * SQRT(390*250) as sigma_intraday 
        FROM  sp500_iex_intraday
        GROUP BY symbol'''
pd.read_sql(q, conn)

Wall time: 258 ms


,symbol,sigma_intraday
0,A,37.119589
1,AAL,41.106203
2,AAP,33.672927
3,AAPL,29.697883
4,ABBV,59.048038
...,...,...
498,YUM,38.338165
499,ZBH,43.586759
500,ZBRA,73.928335
501,ZION,68.026810


# JOIN Ejemplo

## Ejemplo long/short vega

Condiciones


* Que presenten balances en los proximos 7 dias
* Que tengan Sigma diaria de las ultimas 40 ruedas menor/mayor a la de las ultimas 120 (mean reversion)
* Que la volatilidad anualizada HF, PICO en ultimos 15 dias, >+30% / <-30%  al sigma 250 ruedas
* Que la volatilidad anualizada de las velas de 1min del ultimo dia sea  >+10% / <-10% al sigma 40 ruedas


Screener para los tickers que cumplan las condiciones:

* Traer para cada ticker, los 5 contratos de menor/mayor VI
* Traer para cada ticker, los 5 contratos de mayor vega

Y si cambio el ejemplo el punto 2-- En vez de buscar un mean reversion buscar una aceleracion 40 vs 120?

## Creacion de vistas para parametros de las condiciones

In [74]:
hoy = datetime.date.today().isoformat()

fecha_limite = datetime.date.today() + datetime.timedelta(days=7)
fecha_limite = fecha_limite.isoformat()

q = f'''CREATE VIEW IF NOT EXISTS bces_7d AS 
        SELECT symbol, EarningsDate FROM sp500_earnings
        WHERE EarningsDate < "{fecha_limite}" AND EarningsDate > "{hoy}"   
    '''

conn.execute(q)

In [75]:
q = f'''CREATE VIEW IF NOT EXISTS sigmas_diarios AS 
            SELECT symbol, sigma_40, sigma_120, sigma_250
                FROM sp500_daily 
                WHERE Date =
                    (SELECT MAX(Date) FROM sp500_daily)
    '''

conn.execute(q)

In [76]:
n_dias = 15
q = f'''CREATE VIEW IF NOT EXISTS sigmas_high_freq_{n_dias}d AS 
                    SELECT symbol, round(STDDEV(variacion)*SQRT(250*78),2) AS sigma_hf,
                        MAX(sigma_1d) AS hf_pico_r1d , 
                        MAX(sigma_5d) AS hf_pico_r5d
                    FROM sp500_5m 
                    WHERE Datetime > DATE_SUB(NOW(), INTERVAL {n_dias} day) 
                    GROUP BY symbol'''

conn.execute(q)

In [77]:
q = '''CREATE VIEW IF NOT EXISTS sigmas_1min AS 
            SELECT symbol, STDDEV(pctChange) * SQRT(390*250) as sigma_intraday 
            FROM  sp500_iex_intraday
            GROUP BY symbol'''

conn.execute(q)

## Condiciones

In [86]:
%%time
q = '''SELECT * FROM bces_7d AS b
            JOIN sigmas_diarios AS sd
                ON b.symbol = sd.symbol
            JOIN sigmas_high_freq_15d AS shf
                ON sd.symbol = shf.symbol
            JOIN sigmas_1min AS s1m
                ON shf.symbol = s1m.symbol
            
            WHERE sd.sigma_40 < sd.sigma_120
            AND  shf.hf_pico_r1d > sd.sigma_250 * 1.3
            AND s1m.sigma_intraday > sd.sigma_40 * 1.1
    '''

long_vega = pd.read_sql(q, conn)
long_vega

Wall time: 1.4 s


,symbol,EarningsDate,symbol,sigma_40,sigma_120,sigma_250,symbol,sigma_hf,hf_pico_r1d,hf_pico_r5d,symbol,sigma_intraday
0,ABC,2020-11-05,ABC,24.06,26.73,41.55,ABC,37.94,54.86,103.30,ABC,41.550608
1,CAH,2020-11-05,CAH,29.45,33.97,44.32,CAH,39.83,63.77,110.59,CAH,42.691738
2,DHI,2020-11-10,DHI,40.91,43.80,57.10,DHI,53.23,106.39,143.81,DHI,53.070031
3,DISCA,2020-11-05,DISCA,39.56,44.26,49.40,DISCA,42.99,65.83,109.49,DISCA,50.772728
4,EA,2020-11-05,EA,27.23,27.98,35.22,EA,32.90,78.45,94.10,EA,38.611689
5,FLS,2020-11-05,FLS,41.77,54.26,59.80,FLS,52.60,99.28,135.00,FLS,68.498973
6,HFC,2020-11-05,HFC,52.29,60.35,71.29,HFC,66.39,101.93,160.97,HFC,73.209484
7,LIN,2020-11-05,LIN,26.26,27.93,38.13,LIN,31.16,51.56,84.50,LIN,29.202806
8,MNST,2020-11-05,MNST,25.27,25.45,35.60,MNST,33.69,51.74,88.52,MNST,47.137246
9,MYL,2020-11-06,MYL,35.18,38.60,45.03,MYL,40.70,61.46,103.09,MYL,48.559479


In [89]:
long_vega_tickers = tuple(long_vega.iloc[:,0])

## Screener

In [90]:
%%time

screener = {}
for ticker in long_vega_tickers:
    q = f'''SELECT opc.* FROM sp500_opciones opc
                JOIN sigmas_diarios AS sd
                    ON opc.symbol = sd.symbol

                WHERE opc.symbol = "{ticker}"
                    AND opc.ITM = 0
                    AND opc.IV <> 'None'
                
                ORDER BY opc.IV ASC
                LIMIT 5
    '''
    screener[ticker] = pd.read_sql(q, conn)

Wall time: 4.68 s


In [91]:
screener['ABC']

,index,symbol_opc,Strike,TTM,Type,Bid,Ask,Last,IV,OpenInt,Theor,delta,gamma,theta,vega,rho,ITM,symbol,id
0,333,ABC_011521C130,130.0,72,CALL,0.25,0.55,0.30,29.232,741,0.4,0.069,0.009,-0.012,0.063,0.014,0,ABC,64234
1,361,ABC_021921C140,140.0,107,CALL,0.20,0.45,1.00,29.449,5,0.325,0.049,0.006,-0.008,0.058,0.014,0,ABC,64262
2,362,ABC_021921C145,145.0,107,CALL,0.05,0.35,0.76,29.453,30,0.2,0.032,0.004,-0.005,0.041,0.009,0,ABC,64263
3,334,ABC_011521C135,135.0,72,CALL,0.10,0.35,0.80,29.459,5,0.225,0.041,0.006,-0.008,0.042,0.008,0,ABC,64235
4,360,ABC_021921C135,135.0,107,CALL,0.45,0.70,0.35,29.804,3,0.575,0.079,0.008,-0.011,0.085,0.023,0,ABC,64261
